In [ ]:
import random
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import requests
import os
import scipy.sparse as sp

##########################
url = 'https://api.census.gov/data/2019/pep/population?get=NAME&for=county:*'

response = requests.get(url)
data = response.json()


county_count = len(data) - 1  # Subtracting 1 to exclude the header row
print(f"Number of counties: {county_count}")
def get_edge_road_types(road_graph):
    road_types = set()

    for _, _, data in road_graph.edges(data=True):
        if 'highway' in data:
            highway = data['highway']
            if isinstance(highway, list):
                road_types.update(highway)
            else:
                road_types.add(highway)

    return road_types


def filter_residential_edges(road_graph):
    filtered_graph = road_graph.copy()

    for u, v, key, data in road_graph.edges(keys=True, data=True):
        if 'highway' in data and (data['highway'] == 'residential'
                                 or data['highway'] == 'trunk' or data['highway'] == 'trunk_link'
                                 or data['highway'] == 'alley' or data['highway'] == 'unclassified'
                                 or data['highway'] == 'living_street'):
            filtered_graph.remove_edge(u, v, key=key)

    isolated_nodes = [node for node in filtered_graph.nodes if filtered_graph.degree[node] == 0]
    filtered_graph.remove_nodes_from(isolated_nodes)

    return filtered_graph

#######################
def get_county_names():
    response = requests.get('https://api.census.gov/data/2019/pep/population?get=NAME&for=county:*')
    data = response.json()

    county_names = [county[0] for county in data[1:]]
    return county_names

x = # select a minimum number of nodes to avoid empty graphs.
def extract_road_graphs():
    county_names = get_county_names()
    road_graphs = []

    for county_name in county_names:
        print(f"Extracting road graph for {county_name}...")
        try:
            # Retrieve the road network of the county using osmnx
            G = ox.graph_from_place(county_name, network_type='drive')
            if G.number_of_nodes() > x:
                filtered_graph = filter_residential_edges(G)
                road_graphs.append((filtered_graph, county_name))
                
            else:
                print(f"Skipping {county_name}: Road graph is empty")
                continue
        except (ValueError, nx.NetworkXPointlessConcept) as e:
            print(f"Skipping {county_name}: {str(e)}")
            continue
    


    return road_graphs


###################
save_directory = r""  # Update with your desired save directory
road_graphs = extract_road_graphs()

# Create a new MultiDiGraph to store all the road graphs
combined_graph = nx.MultiDiGraph()


for i, (road_graph, county_name) in enumerate(road_graphs):
    # print(f"Road graph {i+1} edges road types:")
    road_types = get_edge_road_types(road_graph)
    for road_type in road_types:
        print(road_type)
    print()

    # Filter and remove residential edges
    filtered_graph = filter_residential_edges(road_graph)
    
    # Save the road graph
    road_graph_save_path = os.path.join(save_directory, f"{county_name}_road.graphml")
    nx.write_graphml(road_graph, road_graph_save_path)

    # Save the filtered graph
    filtered_graph_save_path = os.path.join(save_directory, f"{county_name}_filtered.graphml")
    nx.write_graphml(filtered_graph, filtered_graph_save_path)

################
import pickle

save_directory = r""  # Update with your desired save directory

# Save each road graph as a separate file
for i, (road_graph, county_name) in enumerate(road_graphs):
    # Construct the file path
    file_path = os.path.join(save_directory, f"{county_name}_road_graph.pkl")

    # Save the road graph using pickle
    with open(file_path, 'wb') as f:
        pickle.dump(road_graph, f)

    print(f"Road graph for {county_name} saved successfully.")
    
#################
import pickle

save_directory = r""  # Update with your desired save directory

# Save each road graph as a separate file
for i, (filtered_graph, county_name) in enumerate(road_graphs):
    # Construct the file path
    file_path = os.path.join(save_directory, f"{county_name}_road_graph.pkl")

    # Save the road graph using pickle
    with open(file_path, 'wb') as f:
        pickle.dump(road_graph, f)

    print(f"Road graph for {county_name} saved successfully.")